In [ ]:
from typing import Tuple, List, Set
from analysis.task5 import query_probes_with_identity, get_identities_with_lowest_and_highest_ap_k
from analysis.task1 import add_identities

import os
os.chdir('./..')

identified_embeddings = add_identities()

# Identity,
samples = query_probes_with_identity()

/home/random01/anaconda3/envs/ironclad-design/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [ ]:
worst_performing_identities, best_performing_identities, lowest_map, highest_map = get_identities_with_lowest_and_highest_ap_k([(x,y) for x,y,_ in samples])
print(f"Worst Performing Identities (map = {lowest_map}):", worst_performing_identities)
print(f"Best Performing Identities (map = {highest_map}):", best_performing_identities)

worst_performing_identities: Set[str] = set(worst_performing_identities)
best_performing_identities: Set[str] = set(best_performing_identities)

distances: List[Tuple[str, List[float]]] = [(x, z) for x,_,z in samples]

In [ ]:
samples[0]